### Alkalinity and Hardness Relationships

WAC resins are particularly effective for removing temporary hardness (carbonate hardness):

In [ ]:
# Calculate hardness fractions
feed_ions = ion_tracking.get('feed_mg_l', {})

# Water chemistry calculations
ca_mg_l = feed_ions.get('Ca_2+', 0)
mg_mg_l = feed_ions.get('Mg_2+', 0)
hco3_mg_l = feed_ions.get('HCO3_-', 0)

# Convert to CaCO3 equivalents
total_hardness = ca_mg_l * 2.5 + mg_mg_l * 4.1
total_alkalinity = hco3_mg_l * 0.82  # HCO3 to CaCO3

# Temporary vs permanent hardness
temp_hardness = min(total_hardness, total_alkalinity)
perm_hardness = max(0, total_hardness - total_alkalinity)

hardness_data = [
    ['Total Hardness', f"{total_hardness:.1f}", 'mg/L as CaCO₃'],
    ['Total Alkalinity', f"{total_alkalinity:.1f}", 'mg/L as CaCO₃'],
    ['Temporary Hardness', f"{temp_hardness:.1f}", 'mg/L as CaCO₃'],
    ['Permanent Hardness', f"{perm_hardness:.1f}", 'mg/L as CaCO₃'],
    ['Temp/Total Ratio', f"{(temp_hardness/total_hardness*100) if total_hardness > 0 else 0:.1f}", '%']
]

df_hardness = pd.DataFrame(hardness_data, columns=['Parameter', 'Value', 'Units'])
display(df_hardness.style.set_caption('Hardness Distribution Analysis'))

In [ ]:
# WAC effectiveness assessment
if temp_hardness / total_hardness > 0.7:
    print("✓ High temporary hardness fraction - WAC is well-suited")
    print(f"  WAC can effectively remove {temp_hardness:.1f} mg/L of the {total_hardness:.1f} mg/L total hardness")
elif temp_hardness / total_hardness > 0.5:
    print("✓ Moderate temporary hardness - WAC is suitable")
    print(f"  WAC will remove {temp_hardness:.1f} mg/L, SAC may be needed for remaining {perm_hardness:.1f} mg/L")
else:
    print("⚠️ Low temporary hardness fraction - consider SAC or WAC+SAC combination")
    print(f"  WAC limited to {temp_hardness:.1f} mg/L removal")

# H-form specific considerations
if 'WAC_H' in resin_info.get('code', ''):
    co2_generated = temp_hardness * 0.44  # mg/L CO2 from alkalinity conversion
    print(f"\n⚠️ H-form WAC will generate approximately {co2_generated:.1f} mg/L CO₂")
    print("  CO₂ degassing will be required for downstream RO protection")